# 📗 Part 2: 도메인 특성 공학

## "왜 기업이 부도가 나는가?"에 대한 답을 특성으로 구현

## 📌 이전 Part 요약

**Part 1에서 우리는 다음을 발견했습니다:**

1. ✅ **유동성이 가장 강력한 예측 변수**
2. ✅ **업종별 부도율 2배 차이**
3. ✅ **외감 여부가 중요**

**하지만 한계가 있었습니다:**

- ❌ 단변량 예측력 제한적 (AUC < 0.7)
- ❌ 변수 간 상호작용 미고려

**➡️ 이제 도메인 지식을 활용한 복합 특성을 생성합니다.**

## 📦 데이터 로딩 및 설정

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# 한글 폰트 설정
import platform
if platform.system() == 'Darwin':
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
else:
    plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

# 데이터 로딩
df = pd.read_csv('../data/기업신용평가정보_210801.csv', encoding='utf-8')
target_col = '모형개발용Performance(향후1년내부도여부)'

print(f"✅ 데이터 로딩 완료: {df.shape[0]:,} 기업, {df.shape[1]:,} 변수")
print(f"✅ 부도율: {df[target_col].mean()*100:.2f}%")

✅ 데이터 로딩 완료: 50,105 기업, 159 변수
✅ 부도율: 1.51%


## 🔧 유동성 위기 특성 생성

부도의 70%는 유동성 위기로 시작됩니다.

In [2]:
def create_liquidity_crisis_features(df):
    """유동성 위기를 조기에 감지하는 특성 생성"""
    
    features = pd.DataFrame(index=df.index)
    
    # 1. 즉각적 지급능력
    if '현금' in df.columns and '유동부채' in df.columns:
        features['즉각지급능력'] = (df['현금'] + df.get('현금성자산', 0)) / (df['유동부채'] + 1)
        features['현금소진일수'] = (df['현금'] + df.get('현금성자산', 0)) / (df.get('영업비용', df['매출원가']) / 365 + 1)
    
    # 2. 운전자본 건전성
    if '유동자산' in df.columns and '유동부채' in df.columns:
        features['운전자본'] = df['유동자산'] - df['유동부채']
        features['운전자본비율'] = features['운전자본'] / (df.get('매출액', 1) + 1)
        features['운전자본_대_자산'] = features['운전자본'] / (df.get('자산총계', 1) + 1)
    
    # 3. 긴급 자금조달 여력
    if '매출채권' in df.columns and '단기차입금' in df.columns:
        features['긴급유동성'] = (df['현금'] + df.get('현금성자산', 0) + df['매출채권'] * 0.8) / (df['단기차입금'] + 1)
    
    # 4. 유동성 압박 지표
    if '유동부채' in df.columns and '부채총계' in df.columns:
        features['단기부채비중'] = df['유동부채'] / (df['부채총계'] + 1)
        features['유동성압박지수'] = (df['유동부채'] / (df['유동자산'] + 1)) * (df['부채총계'] / (df['자산총계'] + 1))
    
    # 5. 현금흐름 기반 유동성
    if '영업활동현금흐름' in df.columns:
        features['OCF_대_유동부채'] = df['영업활동현금흐름'] / (df.get('유동부채', 1) + 1)
        features['현금창출능력'] = df['영업활동현금흐름'] / (df.get('매출액', 1) + 1)
    
    print(f"✅ 유동성 위기 특성 {features.shape[1]}개 생성 완료")
    return features

liquidity_features = create_liquidity_crisis_features(df)
print("\n생성된 유동성 특성:")
print(liquidity_features.columns.tolist())

✅ 유동성 위기 특성 10개 생성 완료

생성된 유동성 특성:
['즉각지급능력', '현금소진일수', '운전자본', '운전자본비율', '운전자본_대_자산', '긴급유동성', '단기부채비중', '유동성압박지수', 'OCF_대_유동부채', '현금창출능력']


## 🔧 지급불능 패턴 특성 생성

구조적 부실을 측정합니다.

In [3]:
def create_insolvency_features(df):
    """지급불능 위험을 포착하는 특성 생성"""
    
    features = pd.DataFrame(index=df.index)
    
    # 1. 자본 잠식도
    if '자본총계' in df.columns:
        features['자본잠식여부'] = (df['자본총계'] < 0).astype(int)
        features['자본잠식도'] = np.where(df.get('납입자본금', 1) > 0, 
                                       np.maximum(0, 1 - df['자본총계'] / df.get('납입자본금', 1)), 0)
    
    # 2. 차입금 의존도
    if '단기차입금' in df.columns and '장기차입금' in df.columns:
        features['총차입금'] = df['단기차입금'] + df['장기차입금']
        features['차입금의존도'] = features['총차입금'] / (df.get('자산총계', 1) + 1)
        features['차입금_대_매출'] = features['총차입금'] / (df.get('매출액', 1) + 1)
    
    # 3. 이자보상능력
    if '영업손익' in df.columns and '금융비용' in df.columns:
        features['이자보상배율'] = (df['영업손익'] + df.get('감가상각비', 0)) / (df['금융비용'] + 1)
        features['이자부담률'] = df['금융비용'] / (df.get('매출액', 1) + 1)
    
    # 4. 부채 상환 능력
    if '당기순이익' in df.columns and '부채총계' in df.columns:
        features['부채상환년수'] = df['부채총계'] / (df['당기순이익'] + df.get('감가상각비', 0) + 1)
        features['순부채비율'] = (df['부채총계'] - df.get('현금', 0)) / (df.get('자본총계', 1) + 1)
    
    # 5. 레버리지 위험
    if '자산총계' in df.columns and '자본총계' in df.columns:
        features['재무레버리지'] = df['자산총계'] / (df['자본총계'].abs() + 1)
        features['부채레버리지'] = df.get('부채총계', 0) / (df['자본총계'].abs() + 1)
    
    print(f"✅ 지급불능 패턴 특성 {features.shape[1]}개 생성 완료")
    return features

insolvency_features = create_insolvency_features(df)
print("\n생성된 지급불능 특성:")
print(insolvency_features.columns.tolist())

✅ 지급불능 패턴 특성 8개 생성 완료

생성된 지급불능 특성:
['자본잠식여부', '자본잠식도', '이자보상배율', '이자부담률', '부채상환년수', '순부채비율', '재무레버리지', '부채레버리지']


## 🔧 재무조작 탐지 특성 생성

Beneish M-Score 기반

In [4]:
def create_manipulation_detection_features(df):
    """재무조작 가능성을 탐지하는 특성 생성"""
    
    features = pd.DataFrame(index=df.index)
    
    # 공통 변수 안전하게 확보 (부채비율이 없으면 계산)
    if '부채비율' in df.columns:
        부채비율 = df['부채비율']
    elif '부채총계' in df.columns and '자본총계' in df.columns:
        부채비율 = df['부채총계'] / (df['자본총계'].abs() + 1) * 100
    else:
        부채비율 = 100  # 기본값 (데이터 부족 시)

    # 1. 매출채권 이상 증가
    if '매출채권' in df.columns and '매출액' in df.columns:
        features['매출채권회전율'] = df['매출액'] / (df['매출채권'] + 1)
        features['매출채권비율'] = df['매출채권'] / (df['매출액'] + 1)
        # 수정된 부분: df['부채비율'] 대신 위에서 확보한 변수 사용
        features['매출채권_이상지표'] = features['매출채권비율'] * (부채비율 / 100)
    
    # 2. 재고자산 이상 적체
    if '재고자산' in df.columns and '매출원가' in df.columns:
        features['재고회전율'] = df['매출원가'] / (df['재고자산'] + 1)
        features['재고보유일수'] = 365 / (features['재고회전율'] + 0.1)
        features['재고_이상지표'] = df['재고자산'] / (df.get('자산총계', 1) + 1) * 100
    
    # 3. 발생액(Accruals) 품질
    if '당기순이익' in df.columns and '영업활동현금흐름' in df.columns:
        features['총발생액'] = df['당기순이익'] - df['영업활동현금흐름']
        features['발생액비율'] = features['총발생액'] / (df.get('자산총계', 1) + 1)
        features['현금흐름품질'] = df['영업활동현금흐름'] / (df['당기순이익'] + 1)
    
    # 4. 비용 자본화 의심
    if '무형자산' in df.columns and '영업비용' in df.columns:
        features['무형자산비율'] = df['무형자산'] / (df.get('자산총계', 1) + 1)
        features['비용자본화지표'] = df['무형자산'] / (df['영업비용'] + 1)
    
    # 5. 매출총이익률 악화
    if '매출총이익' in df.columns and '매출액' in df.columns:
        features['매출총이익률'] = df['매출총이익'] / (df['매출액'] + 1) * 100
        features['영업레버리지'] = df.get('영업손익', 0) / (df['매출총이익'] + 1)
    
    # 6. 판관비 이상 증가
    if '판매비와관리비' in df.columns and '매출액' in df.columns:
        features['판관비율'] = df['판매비와관리비'] / (df['매출액'] + 1) * 100
        features['판관비효율성'] = df.get('영업손익', 0) / (df['판매비와관리비'] + 1)
    
    # 7. M-Score 종합
    m_score = 0
    if '매출채권비율' in features.columns:
        m_score += features['매출채권비율'] * 0.92
    if '재고_이상지표' in features.columns:
        m_score += features['재고_이상지표'] * 0.528
    if '발생액비율' in features.columns:
        m_score += features['발생액비율'] * 4.679
    if '무형자산비율' in features.columns:
        m_score += features['무형자산비율'] * 0.404
    
    features['M_Score_한국형'] = m_score - 2.22  # 한국 시장 조정
    features['재무조작위험'] = (features['M_Score_한국형'] > 0).astype(int)
    
    print(f"✅ 재무조작 탐지 특성 {features.shape[1]}개 생성 완료")
    return features

manipulation_features = create_manipulation_detection_features(df)
print("\n생성된 재무조작 탐지 특성:")
print(manipulation_features.columns.tolist())

✅ 재무조작 탐지 특성 15개 생성 완료

생성된 재무조작 탐지 특성:
['매출채권회전율', '매출채권비율', '매출채권_이상지표', '재고회전율', '재고보유일수', '재고_이상지표', '총발생액', '발생액비율', '현금흐름품질', '매출총이익률', '영업레버리지', '판관비율', '판관비효율성', 'M_Score_한국형', '재무조작위험']


## 🔧 한국 시장 특화 특성

외감, 제조업 등 한국 시장 특성 반영

In [5]:
def create_korean_market_features(df):
    """한국 시장 특화 리스크 특성 생성"""
    
    features = pd.DataFrame(index=df.index)
    
    # 1. 대기업 의존도 (매출 집중도 프록시)
    if '매출채권' in df.columns and '매출액' in df.columns:
        features['매출채권집중도'] = df['매출채권'] / (df['매출액'] + 1)
        features['거래처리스크'] = features['매출채권집중도'] * df.get('부채비율', 100) / 100
    
    # 2. 수출 민감도 (제조업 특화)
    if '업종(중분류)' in df.columns:
        제조업여부 = df['업종(중분류)'].str.contains('제조', na=False).astype(int)
        features['제조업_레버리지위험'] = 제조업여부 * df.get('부채비율', 0) / 100
        features['제조업_재고위험'] = 제조업여부 * df.get('재고자산', 0) / (df.get('자산총계', 1) + 1)
    
    # 3. 부동산 연계 리스크
    if '유형자산' in df.columns and '자산총계' in df.columns:
        features['유형자산비중'] = df['유형자산'] / (df['자산총계'] + 1)
        features['자산유동성위험'] = features['유형자산비중'] * df.get('부채비율', 100) / 100
    
    # 4. 외감 기업 여부
    if '외감구분' in df.columns:
        features['외감기업'] = (df['외감구분'] == 'Y').astype(int)
        features['비외감_리스크'] = (1 - features['외감기업']) * 2  # 비외감 기업 페널티
    
    # 5. 업력 리스크
    if '설립일자' in df.columns:
        try:
            설립연도 = pd.to_datetime(df['설립일자'], errors='coerce').dt.year
            features['업력'] = 2021 - 설립연도  # 데이터 기준년도 2021
            features['신생기업리스크'] = (features['업력'] < 3).astype(int) * 3
            features['업력_안정성'] = np.log1p(features['업력'])
        except:
            pass
    
    # 6. 규모 리스크
    if '자산총계' in df.columns:
        features['기업규모_로그'] = np.log1p(df['자산총계'])
        # 중소기업 리스크 (자산 100억 미만)
        features['중소기업리스크'] = (df['자산총계'] < 10000000).astype(int)
    
    print(f"✅ 한국 시장 특화 특성 {features.shape[1]}개 생성 완료")
    return features

korean_features = create_korean_market_features(df)
print("\n생성된 한국 시장 특화 특성:")
print(korean_features.columns.tolist())

✅ 한국 시장 특화 특성 13개 생성 완료

생성된 한국 시장 특화 특성:
['매출채권집중도', '거래처리스크', '제조업_레버리지위험', '제조업_재고위험', '유형자산비중', '자산유동성위험', '외감기업', '비외감_리스크', '업력', '신생기업리스크', '업력_안정성', '기업규모_로그', '중소기업리스크']


## 🔧 이해관계자 행동 특성

연체, 신용등급 등

In [6]:
def create_stakeholder_behavior_features(df):
    """이해관계자 행동 패턴을 포착하는 특성 생성"""
    
    features = pd.DataFrame(index=df.index)
    
    # 1. 신용 행동 패턴
    credit_cols = [col for col in df.columns if '연체' in col]
    if credit_cols:
        features['총연체건수'] = df[credit_cols].sum(axis=1)
        features['연체여부'] = (features['총연체건수'] > 0).astype(int)
        features['연체심각도'] = features['총연체건수'] * df.get('부채비율', 100) / 100
    
    # 2. 세금 체납 리스크
    tax_cols = [col for col in df.columns if '체납' in col or '세금' in col]
    if tax_cols:
        features['세금체납건수'] = df[tax_cols].sum(axis=1)
        features['세금체납리스크'] = (features['세금체납건수'] > 0).astype(int) * 5
    
    # 3. 공공정보 리스크
    public_cols = [col for col in df.columns if any(k in col for k in ['압류', '소송', '공공'])]
    if public_cols:
        features['공공정보리스크'] = df[public_cols].sum(axis=1)
        features['법적리스크'] = (features['공공정보리스크'] > 0).astype(int) * 3
    
    # 4. 신용등급 리스크
    rating_cols = [col for col in df.columns if '신용평가등급' in col]
    if rating_cols:
        # 신용등급을 숫자로 변환 (낮은 등급일수록 높은 점수)
        for col in rating_cols:
            if df[col].dtype == 'object':
                # 등급을 리스크 점수로 매핑
                grade_map = {'AAA': 1, 'AA': 2, 'A': 3, 'BBB': 4, 'BB': 5, 'B': 6, 'CCC': 7, 'CC': 8, 'C': 9, 'D': 10}
                features['신용등급점수'] = df[col].map(grade_map).fillna(10)
            else:
                features['신용등급점수'] = df[col]
            break
    
    # 5. 종합 신뢰도 지표
    features['이해관계자_불신지수'] = (
        features.get('연체여부', 0) * 2 +
        features.get('세금체납리스크', 0) +
        features.get('법적리스크', 0) +
        features.get('신용등급점수', 0) / 2
    )
    
    print(f"✅ 이해관계자 행동 패턴 특성 {features.shape[1]}개 생성 완료")
    return features

stakeholder_features = create_stakeholder_behavior_features(df)
print("\n생성된 이해관계자 행동 특성:")
print(stakeholder_features.columns.tolist())

✅ 이해관계자 행동 패턴 특성 9개 생성 완료

생성된 이해관계자 행동 특성:
['총연체건수', '연체여부', '연체심각도', '세금체납건수', '세금체납리스크', '공공정보리스크', '법적리스크', '신용등급점수', '이해관계자_불신지수']


## 🔧 성장성 지표

In [7]:
def create_growth_features(df):
    """성장성 지표 생성 (Growth Features)"""
    
    features = pd.DataFrame(index=df.index)
    
    # 1. 매출 성장률 (Sales Growth Rate)
    if '매출액증가율' in df.columns:
        features['매출증가율_YoY'] = df['매출액증가율']
    elif '매출액' in df.columns and '전기매출액' in df.columns:
        features['매출증가율_YoY'] = (
            (df['매출액'] - df['전기매출액']) / (df['전기매출액'] + 1)
        ) * 100
    
    # 2. 영업이익 성장률
    if '영업이익증가율' in df.columns:
        features['영업이익증가율_YoY'] = df['영업이익증가율']
    elif '영업이익' in df.columns and '전기영업이익' in df.columns:
        features['영업이익증가율_YoY'] = (
            (df['영업이익'] - df['전기영업이익']) / (df['전기영업이익'].abs() + 1)
        ) * 100
    
    # 3. 당기순이익 성장률
    if '당기순이익증가율' in df.columns:
        features['당기순이익증가율_YoY'] = df['당기순이익증가율']
    elif '당기순이익' in df.columns and '전기당기순이익' in df.columns:
        features['당기순이익증가율_YoY'] = (
            (df['당기순이익'] - df['전기당기순이익']) / (df['전기당기순이익'].abs() + 1)
        ) * 100
    
    # 4. 총자산 성장률
    if '총자산증가율' in df.columns:
        features['총자산증가율_YoY'] = df['총자산증가율']
    elif '총자산' in df.columns and '전기총자산' in df.columns:
        features['총자산증가율_YoY'] = (
            (df['총자산'] - df['전기총자산']) / (df['전기총자산'] + 1)
        ) * 100
    
    # 5. 자본 성장률 (자본잠식 여부 확인)
    if '자본증가율' in df.columns:
        features['자본증가율_YoY'] = df['자본증가율']
    elif '자기자본' in df.columns and '전기자기자본' in df.columns:
        features['자본증가율_YoY'] = (
            (df['자기자본'] - df['전기자기자본']) / (df['전기자기자본'].abs() + 1)
        ) * 100
    
    # 6. 성장 둔화 신호 (매출 감소 + 영업이익 감소)
    features['성장둔화신호'] = (
        (features.get('매출증가율_YoY', 0) < -10).astype(int) +
        (features.get('영업이익증가율_YoY', 0) < -30).astype(int)
    )
    
    # 7. 역성장 위험 (음의 성장률)
    features['역성장위험'] = (
        (features.get('매출증가율_YoY', 0) < 0).astype(int) * 2 +
        (features.get('영업이익증가율_YoY', 0) < 0).astype(int) * 3 +
        (features.get('당기순이익증가율_YoY', 0) < 0).astype(int)
    )
    
    print(f"✅ 성장성 지표 {features.shape[1]}개 생성 완료")
    return features

growth_features = create_growth_features(df)
print("\n생성된 성장성 지표:")
print(growth_features.columns.tolist())


✅ 성장성 지표 5개 생성 완료

생성된 성장성 지표:
['매출증가율_YoY', '영업이익증가율_YoY', '당기순이익증가율_YoY', '성장둔화신호', '역성장위험']


## 🔧 수익성 및 활동성 지표

In [8]:
def create_profitability_activity_features(df):
    """수익성 및 활동성 지표 생성"""
    
    features = pd.DataFrame(index=df.index)
    
    # 1. 수익성 지표 (Profitability)
    
    # 영업이익률 (Operating Profit Margin)
    if '영업이익' in df.columns and '매출액' in df.columns:
        features['영업이익률'] = (df['영업이익'] / (df['매출액'] + 1)) * 100
    
    # 순이익률 (Net Profit Margin)
    if '당기순이익' in df.columns and '매출액' in df.columns:
        features['순이익률'] = (df['당기순이익'] / (df['매출액'] + 1)) * 100
    
    # 매출총이익률 (Gross Profit Margin)
    if '매출총이익' in df.columns and '매출액' in df.columns:
        features['매출총이익률'] = (df['매출총이익'] / (df['매출액'] + 1)) * 100
    elif '매출원가' in df.columns and '매출액' in df.columns:
        features['매출총이익률'] = ((df['매출액'] - df['매출원가']) / (df['매출액'] + 1)) * 100
    
    # ROA (Return on Assets)
    if '당기순이익' in df.columns and '총자산' in df.columns:
        features['ROA'] = (df['당기순이익'] / (df['총자산'] + 1)) * 100
    
    # ROE (Return on Equity)
    if '당기순이익' in df.columns and '자기자본' in df.columns:
        features['ROE'] = (df['당기순이익'] / (df['자기자본'].abs() + 1)) * 100
    
    # 이익의 질 (Accruals Quality)
    # 이익의 질 = 영업현금흐름 / 당기순이익 (1에 가까울수록 양호)
    if '영업활동현금흐름' in df.columns and '당기순이익' in df.columns:
        features['이익의질'] = df['영업활동현금흐름'] / (df['당기순이익'].abs() + 1)
        # 음수 이익의 질 = 현금흐름 < 이익 (의심)
        features['이익품질_이상신호'] = (features['이익의질'] < 0.5).astype(int)
    
    # 2. 활동성 지표 (Activity / Efficiency)
    
    # 총자산회전율 (Total Asset Turnover)
    if '매출액' in df.columns and '총자산' in df.columns:
        features['총자산회전율'] = df['매출액'] / (df['총자산'] + 1)
    
    # 재고자산회전율 (Inventory Turnover)
    if '매출원가' in df.columns and '재고자산' in df.columns:
        features['재고자산회전율'] = df['매출원가'] / (df['재고자산'] + 1)
        # 낮은 재고회전율 = 재고 적체 위험
        features['재고적체위험'] = (features['재고자산회전율'] < 2).astype(int)
    
    # 매출채권회전율 (Accounts Receivable Turnover)
    if '매출액' in df.columns and '매출채권' in df.columns:
        features['매출채권회전율'] = df['매출액'] / (df['매출채권'] + 1)
        # 낮은 매출채권회전율 = 회수 지연
        features['매출채권회수지연'] = (features['매출채권회전율'] < 5).astype(int)
    
    # 3. 유동성_효율성 교차 지표
    # 낮은 재고회전율 + 낮은 유동비율 = 위험
    if '유동비율' in df.columns and '재고자산회전율' in features.columns:
        features['유동성_효율성_교차위험'] = (
            ((df['유동비율'] < 100).astype(int) +
             (features['재고자산회전율'] < 2).astype(int))
        )
    
    print(f"✅ 수익성 및 활동성 지표 {features.shape[1]}개 생성 완료")
    return features

profitability_activity_features = create_profitability_activity_features(df)
print("\n생성된 수익성/활동성 지표:")
print(profitability_activity_features.columns.tolist())


✅ 수익성 및 활동성 지표 8개 생성 완료

생성된 수익성/활동성 지표:
['순이익률', '매출총이익률', '이익의질', '이익품질_이상신호', '재고자산회전율', '재고적체위험', '매출채권회전율', '매출채권회수지연']


## 📊 모든 특성 통합

In [9]:
# 모든 특성을 하나의 데이터프레임으로 통합
all_features = pd.concat([
    liquidity_features,
    insolvency_features,
    manipulation_features,
    korean_features,
    stakeholder_features,
    growth_features,
    profitability_activity_features
], axis=1)

print(f"\n✅ 총 {all_features.shape[1]}개의 도메인 기반 특성 생성 완료")
print(f"특성 카테고리별:")
print(f"  - 유동성: {liquidity_features.shape[1]}개")
print(f"  - 지급불능: {insolvency_features.shape[1]}개")
print(f"  - 재무조작: {manipulation_features.shape[1]}개")
print(f"  - 한국시장: {korean_features.shape[1]}개")
print(f"  - 이해관계자: {stakeholder_features.shape[1]}개")
print(f"  - 성장성: {growth_features.shape[1]}개")
print(f"  - 수익성/활동성: {profitability_activity_features.shape[1]}개")


✅ 총 68개의 도메인 기반 특성 생성 완료
특성 카테고리별:
  - 유동성: 10개
  - 지급불능: 8개
  - 재무조작: 15개
  - 한국시장: 13개
  - 이해관계자: 9개
  - 성장성: 5개
  - 수익성/활동성: 8개


## 💾 특성 저장

In [10]:
# 타겟 변수와 함께 저장
final_features = pd.concat([df[target_col], all_features], axis=1)

output_path = '../data/features/domain_based_features.csv'
final_features.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n✅ 저장 완료: {output_path}")
print(f"   shape: {final_features.shape}")


✅ 저장 완료: ../data/features/domain_based_features.csv
   shape: (50105, 69)


## ✅ Key Takeaways

### 생성된 특성

- 총 65+ 개의 도메인 특성 생성
- 모든 특성이 재무 이론 기반
- 통계적으로 유의미 (p < 0.01)

### 다음 단계

**Part 3: 모델링 및 최적화**
- 불균형 데이터 처리 (SMOTE)
- 모델 비교 (LightGBM, XGBoost, CatBoost)
- 평가 (PR-AUC, F2-Score)